In [1]:
#pip install transformers==4.15.0

In [2]:
!nvidia-smi

/bin/bash: /home/apsisdev/anaconda3/envs/eblict/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Mon Aug 29 14:39:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 30%   51C    P0   111W / 370W |    139MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+-----------

In [3]:
pwd

'/home/apsisdev/mobassir'

In [4]:
ls  '/home/apsisdev/mobassir/'

/bin/bash: /home/apsisdev/anaconda3/envs/eblict/lib/libtinfo.so.6: no version information available (required by /bin/bash)
 aaivu-tamizhi-net-OCR-master/
 Arabic_Fonts_Check/
 banglaSpeech/
 best-transformer-representations.ipynb
 bn_old_testament_of_kjv.csv
 bnsynthtiger/
 bn_translated_tafseer.csv
 data/
'data downloader.ipynb'
 datasets/
 deberta-base.py
 deberta.py
 deberta-train.ipynb
 docUnet.pytorch-master/
 en_bn_bukhari_muslim/
 en_bn_bukhari_muslim.csv
 en_bn_translated_sahih_hadiths.csv
 EN_old_testament_of_kjv.csv
'feedback prize baseline.ipynb'
 fonts/
 get_oof_csv.ipynb
 handwriting_line_generation/
 Handwriting-Transformers-main/
 infer_1.ipynb
 molat_pdf_to_imgs.ipynb
 NID-PORICHOY/
 nllb-200-1.3B/
 nllb-200-3.3B/
 nllb-200-distilled-1.3B/
 nllb-200-distilled-600M/
 nllb_en_bn_translator.ipynb
 nllb_en_bn_translator.py
 nlp_profanity_data_generator.ipynb
 pppm5class.ipynb
'pppm awp.py'
 pppm_awp.py
 profanity/
 pytorch-feedback-deberta-v3-baseline.ipynb
 pytorch_robert

In [5]:
import transformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm

import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import gc
import re
tqdm.pandas()
print(torch.__version__)
transformers.__version__

2022-08-29 14:39:25.279458: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


1.12.1


'4.21.2'

In [6]:
%%time
# !pip install -q transformers

model = AutoModelForSeq2SeqLM.from_pretrained("/home/apsisdev/mobassir/nllb-200-1.3B")
tokenizer = AutoTokenizer.from_pretrained("/home/apsisdev/mobassir/nllb-200-1.3B")

CPU times: user 17.6 s, sys: 3.27 s, total: 20.8 s
Wall time: 12.4 s


In [7]:
%%time

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
translation_pipeline = pipeline('translation', 
                                model=model, 
                                tokenizer=tokenizer, 
                                src_lang="eng_latn",  #arb_Arab,eng_latn
                                tgt_lang='ben_Beng',
                                device = device,
                                max_length = 1024)

cuda:0
CPU times: user 1.35 s, sys: 743 ms, total: 2.1 s
Wall time: 2.58 s


In [8]:
translation_pipeline.device

device(type='cuda', index=0)

In [9]:
en_bn_bukhari_muslim = pd.read_csv('/home/apsisdev/mobassir/en_bn_bukhari_muslim/en_bukhari_muslim.csv')
en_bn_bukhari_muslim.head(2)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,'Abdullah bin al-Zubair bin 'Isa عبد الله بن ا...
1,1,2,Sahih Bukhari,1,2,Revelation - كتاب بدء الوحى,"30355, 20001, 11065, 10511, 53",حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated 'Aisha: ...,'Abdullah bin Yusuf al-Tunaysi عبد الله بن يوس...


In [10]:
%%time

def remove_non_english(mixed_text,only_special = False):
    if(only_special):
        return mixed_text.replace("\'","")
    return re.sub("[^a-zA-Z0-9,']+", " ",str(mixed_text))


en_bn_bukhari_muslim["narrators"]=en_bn_bukhari_muslim.narrators.progress_apply(lambda mixed_text: remove_non_english(mixed_text))
en_bn_bukhari_muslim["chapter"]=en_bn_bukhari_muslim.chapter.progress_apply(lambda mixed_text: remove_non_english(mixed_text))

en_bn_bukhari_muslim["text_en"]=en_bn_bukhari_muslim.text_en.progress_apply(lambda mixed_text: remove_non_english(mixed_text,only_special = True))

en_bn_bukhari_muslim.narrators[0]

  0%|          | 0/14966 [00:00<?, ?it/s]

  0%|          | 0/14966 [00:00<?, ?it/s]

  0%|          | 0/14966 [00:00<?, ?it/s]

CPU times: user 199 ms, sys: 4.62 ms, total: 203 ms
Wall time: 200 ms


"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin 'Uyaynah ,Yahya bin Sa'id al Ansari ,Muhammad bin Ibrahim bin al Harith ,'Alqama bin Waqqas ,'Umar ibn al Khattab "

In [11]:
en_bn_bukhari_muslim.head(1)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators
0,0,1,Sahih Bukhari,1,1,Revelation,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '..."


In [12]:
en_bn_bukhari_muslim.text_en[2]

' Narrated Aisha:                       (the mother of the faithful believers) The commencement of the Divine Inspiration to Allahs Apostle was in the form of good dreams which came true like bright daylight, and then the love of seclusion was bestowed upon him. He used to go in seclusion in the cave of Hira where he used to worship (Allah alone) continuously for many days before his desire to see his family. He used to take with him the journey food for the stay and then come back to (his wife) Khadija to take his food likewise again till suddenly the Truth descended upon him while he was in the cave of Hira. The angel came to him and asked him to read. The Prophet replied, "I do not know how to read." The Prophet added, "The angel caught me (forcefully) and pressed me so hard that I could not bear it any more. He then released me and again asked me to read and I replied, I do not know how to read. Thereupon he caught me again and pressed me a second time till I could not bear it any 

In [13]:
%%time
print(len(en_bn_bukhari_muslim))
sentences = re.split(r'(?<=\w\.)\s', en_bn_bukhari_muslim.text_en[4])
bn = []
for i in range(len(sentences)):
    result = translation_pipeline(sentences[i])
    bn.append(result[0]['translation_text'])
bn = " ".join(bn)
bn = re.sub(' ্ ','',bn)
bn

14966
CPU times: user 5.58 s, sys: 204 ms, total: 5.78 s
Wall time: 6.17 s


'তিনি বলেন, ইবনে জুমাইর বলেনঃ ইবনে আব্বাস আল্লার এই বক্তব্যের ব্যাখ্যা বর্ণনায় ইবনে আব্বাস বলেন, "তোমরা কুরআনের ব্যাপারে তোমাদের জিহ্বাকে না সরিয়ে ফেলো, যাতে তোমরা তা তাড়াতাড়ি করতে পারো।" (৭৫ঃ১৬) তিনি বলেন, "আল্লাহর রসূল (সাঃ) খুব কষ্টে অবতীর্ণ অবতীর্ণ অবতীর্ণ করেন এবং প্রেরণ দিয়ে তাঁর ঠোঁট (দ্রুত) সরিয়ে দেন।" ইবনে আব্বাস তাঁর ঠোঁট সরিয়ে বলেন, "আমি আপনার সামনে আমার ঠোঁট সরিয়ে দিচ্ছি যেমন আল্লাহর রসূল (সাঃ) তাঁর ঠোঁট সরিয়ে দিয়েছিলেন।" তিনি বলেন, "আমি আমার ঠোঁট সরিয়ে দিচ্ছি, যেমন আমি ইবনে আব্বাসকে তাঁর ঠোঁট সরিয়ে দেখেছিলাম।" ইবনে আব্বাস যোগ করেন, "সুতরাং আল্লাহ্ অবতীর্ণ করেছেন, যাতে তোমরা কুরআনের ব্যাপারে তোমাদের জিহ্বাকে না সরিয়ে ফেলো যাতে তোমরা তা তাড়াতাড়ি করতে পারো।" আর আমাদেরই দায়িত্ব তা সংগ্রহ করা এবং তোমার (হে মুহাম্মদ) কে তা পাঠ করার ক্ষমতা দেয়া। আর যখন আমি এটি তোমার কাছে পাঠ করি তখন তুমি এর পাঠ অনুসরণ কর। অতঃপর আমিই তা সুস্পষ্ট করে দেব, অতঃপর আমিই তা পাঠ করব। এর পরে যখনই রসূল (সাঃ) এসেছিলেন, তিনি জিব্রাইলের কথা শুনতেন, অতঃপর যখন তিনি চলে যান তখন তিনি জিব্রাইলের ম

In [14]:
# en_bn_bukhari_muslim['Count'] = en_bn_bukhari_muslim.text_ar.apply(lambda x: len(x.split(" ")))
# en_bn_bukhari_muslim = en_bn_bukhari_muslim.sort_values(['Count'], ascending=False)
# print(en_bn_bukhari_muslim.iloc[0]) #1924

In [15]:
%%time

def en_to_bn_translator(en_text):
    
#     sentences = re.split(r'(?<=\w\.)\s', str(en_text))
    
    sentenceEnders = re.compile('[.!?]')
    sentences = sentenceEnders.split(str(en_text))

    bn = []
    for i in range(len(sentences)):
        result = translation_pipeline(sentences[i])
        bn.append(result[0]['translation_text'])
    res = " ".join(bn)
    res = re.sub(' ্ ','',res)
    return res
    
en_bn_bukhari_muslim["text_bn"]=en_bn_bukhari_muslim.text_en.progress_apply(lambda en_text: en_to_bn_translator(en_text))


  0%|          | 0/14966 [00:00<?, ?it/s]

/home/apsisdev/anaconda3/envs/kaggle/lib/python3.8/site-packages/transformers/pipelines/base.py:1036: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: user 11h 25min 30s, sys: 2min 39s, total: 11h 28min 10s
Wall time: 11h 28min


In [16]:

en_bn_bukhari_muslim.to_csv('./en_bn_bukhari_muslim.csv',index = False)

In [17]:
en_bn_bukhari_muslim

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '...",উমর বিন আল-খাত্তাবের বর্ণনায় উমর বিন আল-খাত্ত...
1,1,2,Sahih Bukhari,1,2,Revelation,"30355, 20001, 11065, 10511, 53",حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated Aisha: ...,"'Abdullah bin Yusuf al Tunaysi ,Imam Maalik ,H...",আয়েশা (রাঃ) এর কাহিনী বর্ণিত হয়েছে: (বিশ্বাস...
2,2,3,Sahih Bukhari,1,3,Revelation,"30399, 20023, 11207, 11013, 10511, 53",حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، ...,Narrated Aisha: (the mo...,"Yahya bin Yahya bin Bukayr ,al Laith bin Sa'd ...",আয়েশা (রাঃ) এর বানীঃ (বিশ্বাসী মুমিনদের মা) আ...
3,3,4,Sahih Bukhari,1,4,Revelation,"11013, 10567, 34",قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أ...,Narrated Jabir bin Abdullah Al-Ansari while t...,"al Zuhri , ,Abu Salama bin 'Abdur Rahman ,Jabi...",বর্ণিত জাবির বিন আবদুল্লাহ আল-আনসারি যখন প্রকা...
4,4,5,Sahih Bukhari,1,5,Revelation,"20040, 20469, 11399, 11050, 17",حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قا...,Narrated Said bin Jubair: ...,"Musa bin Isma'il al Munqari al Tabudhaki ,Wada...","তিনি বলেন, 'আল্লাহর বাণী বর্ণনা করে ইবনে আব্বা..."
...,...,...,...,...,...,...,...,...,...,...,...
14961,36,57,Sahih Muslim,57,7744,The Book of Commentary,"30201, 20345, 11404, 11052, 18",حدثنا أبو بكر بن أبي شيبة، حدثنا علي بن مسهر، ...,Ibn Umar reported that Umar delivered a sermo...,"Abu Bakr bin Abi Shayba ,'Ali bin Ms'hr ,Yahya...","ইবনে উমর (রাঃ) বলেন, উমর আল্লার রসূল (সাঃ) এর ..."
14962,37,57,Sahih Muslim,57,7745,The Book of Commentary,"30216, 20102, 11404, 11052, 18",وحدثنا أبو كريب، أخبرنا ابن إدريس، حدثنا أبو ح...,Ibn Umar reported: I hea...,"Abu Karayb, Muhammad bin al 'Ala' ,'Abdullah b...",ইবনে উমর রিপোর্ট করেছেনঃ আমি উমরকে শুনলাম খাত্...
14963,38,57,Sahih Muslim,57,7746,The Book of Commentary,"30201, 20208, 30123, 20347",وحدثنا أبو بكر بن أبي شيبة، حدثنا إسماعيل ابن ...,This hadith has been transmitted on the same ...,"Abu Bakr bin Abi Shayba ,Ishaq bin Ibrahim bin...",এই হাদিস একই সূত্র থেকে প্রেরিত হয়েছে কিন্তু ...
14964,39,57,Sahih Muslim,57,7747,The Book of Commentary,"30365, 20468, 11454, 11313, 11609",حدثنا عمرو بن زرارة، حدثنا هشيم، عن أبي هاشم، ...,Abu Dharr took an oath that this verse: ...,"'Amr bin Zararah bin Waqid ,Hashaym bin Bashay...","আবু ধারর শপথ নিলেন যে এই আয়াতটিঃ "" এই দুই শত্..."


In [18]:
#dataset link -> https://www.kaggle.com/datasets/mobassir/en-bn-sahih-bukhari-muslim